In [77]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import html5lib

In [ ]:
import numpy as np
seasons = np.arange(9,21)

urls = []
for season in seasons:
    season_url = 'https://en.wikipedia.org/wiki/The_Bachelor_(season_{})'.format(season)
    urls.append(season_url)

In [62]:
def makeSoup(url):
    response=requests.get(url)
    soup=BeautifulSoup(response.content,"lxml")
    return soup

In [141]:
import re

def dataCollector(soup):
    this = makeSoup(soup)
    
    tables = this.findChildren('table')
    table = tables[1]
    
    data   = [[td.text for td in row.select('td')]
             for row in table.findAll('tr')]
    
    
    header = ['name','age', 'hometown', 'occupation', 'elimination']
    body = data[1:]
    cols   =  zip(*body)
    
    tbl_d  = {name:col for name, col in zip(header,cols)}
    
    number = re.findall(r'\d+', soup)
    
    # remove brackets
    num= ''.join(number)

    tbl_d['season'] = [num] * len(dic['age'])
    
    new_names = []
    for name in tbl_d['name']:
        cleaned_name = re.sub(r'\[\d+\]', ' ', name)
        new_names.append(cleaned_name)

    tbl_d['name'] = new_names
    
    return tbl_d

In [152]:
dic = dataCollector('https://en.wikipedia.org/wiki/The_Bachelor_(season_18)')

In [153]:
pd.DataFrame(dic)

ValueError: arrays must all be same length

,age,elimination,hometown,name,occupation,season
0,25,Winner,"Dallas, Texas",Melissa Rycroft,Sales Representative,13
1,24,Runner-up,"Brookfield, Wisconsin",Molly Malaney,Department Store Buyer,13
2,29,Eliminated in week 7,"Peace River, Alberta",Jillian Harris,Interior Designer,13
3,24,Eliminated in week 6,"Carlsbad, California",Naomi Rose Crespo,Flight Attendant,13
4,34,Eliminated in week 5,"Huntsville, Alabama",Stephanie Hogan,Single Mother & Medical Marketing Rep.,13
5,27,Eliminated in week 4,"Marlboro, New Jersey",Lauren Wanger,Teacher,13
6,25,Eliminated in week 4,"Sewickley, Pennsylvania",Megan Parris,Single Mother and Lacrosse Coach,13
7,29,Eliminated in week 4,"Marshall, Missouri",Shannon Bair,Dental Hygienist,13
8,29,Eliminated in week 4,"Blue Island, Illinois","Nicole ""Nikki"" Kaapke",Administrative Assistant,13
9,25,Eliminated in week 3,"Monroe, Connecticut",Erica DeSimone,Account Executive,13
